In [ ]:
!pip install flask-ngrok pyngrok scikit-surprise flask --quiet
!pip install flask pyngrok scikit-surprise --quiet


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.4/154.4 kB 3.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
import pandas as pd
import numpy as np
from surprise import Dataset, Reader, SVD, KNNBasic
from surprise.model_selection import train_test_split
from surprise import accuracy
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from flask import Flask, request, jsonify, render_template_string
from pyngrok import ngrok  # Using pyngrok instead of flask-ngrok

# Loading dataset
df = pd.read_csv("ratings.csv", nrows=100000)
df = df[["userId", "movieId", "rating"]]
df.columns = ["user_id", "movie_id", "rating"]

# Prepare data for Collaborative Filtering
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(df, reader)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Collaborative Filtering (SVD)
svd_model = SVD(n_factors=50)
svd_model.fit(trainset)
svd_predictions = svd_model.test(testset)
print("RMSE (SVD):", accuracy.rmse(svd_predictions))

# Collaborative Filtering (KNN User-Based)
knn_model = KNNBasic(sim_options={'name': 'cosine', 'user_based': True})
knn_model.fit(trainset)
knn_predictions = knn_model.test(testset)
print("RMSE (KNN):", accuracy.rmse(knn_predictions))

# Content-Based Filtering (Using TF-IDF on Movie Titles)
movies_df = pd.read_csv("movies.csv")
movies_df = movies_df[["movieId", "title"]]
movies_df.columns = ["movie_id", "title"]

# Convert titles into numerical vectors
vectorizer = TfidfVectorizer(stop_words='english')
tfidf_matrix = vectorizer.fit_transform(movies_df["title"])
content_similarity = cosine_similarity(tfidf_matrix)

# Hybrid Recommendation System
def hybrid_recommend(user_id, num_recommendations=5):
    all_movies = df["movie_id"].unique()
    rated_movies = df[df["user_id"] == user_id]["movie_id"].tolist()
    unrated_movies = [movie for movie in all_movies if movie not in rated_movies]

    # Collaborative Filtering Predictions
    svd_predictions = [svd_model.predict(user_id, movie) for movie in unrated_movies]
    sorted_svd = sorted(svd_predictions, key=lambda x: x.est, reverse=True)[:num_recommendations]
    top_svd = [(pred.iid, pred.est) for pred in sorted_svd]

    # Content-Based Recommendations
    content_scores = [(movie, content_similarity[movies_df[movies_df["movie_id"] == movie].index[0]].sum()) for movie in unrated_movies]
    sorted_content = sorted(content_scores, key=lambda x: x[1], reverse=True)[:num_recommendations]

    # Combine both methods
    hybrid_results = list(set(top_svd + sorted_content))[:num_recommendations]

    # Convert movie_id to movie titles
    final_recommendations = [
        (movies_df[movies_df["movie_id"] == movie_id]["title"].values[0], score)
        for movie_id, score in hybrid_results
    ]

    return final_recommendations


# Flask App
app = Flask(__name__)

@app.route('/')
def home():
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Movie Recommendation System</title>
        </head>
        <body>
            <h1>Movie Recommendation System</h1>
            <form action="/recommend" method="get">
                <label for="user_id">Enter User ID:</label>
                <input type="number" id="user_id" name="user_id" required>
                <button type="submit">Get Recommendations</button>
            </form>
        </body>
        </html>
    ''')

@app.route('/recommend', methods=['GET'])
def recommend():
    user_id = int(request.args.get('user_id'))
    recommendations = hybrid_recommend(user_id)
    return render_template_string('''
        <!DOCTYPE html>
        <html lang="en">
        <head>
            <meta charset="UTF-8">
            <meta name="viewport" content="width=device-width, initial-scale=1.0">
            <title>Recommendations</title>
        </head>
        <body>
            <h1>Recommended Movies for User {{ user_id }}</h1>
            <ul>
                {% for movie in recommendations %}
                    <li>{{ movie[0] }} - Predicted Score: {{ movie[1] }}</li>
                {% endfor %}
            </ul>
            <a href="/">Go back</a>
        </body>
        </html>
    ''', user_id=user_id, recommendations=recommendations)


# Open an HTTP tunnel using Ngrok
public_url = ngrok.connect(5000).public_url
print(f" * Running on {public_url}")

# Start Flask app
app.run(port=5000)


RMSE: 0.9114
RMSE (SVD): 0.9114459461878137
Computing the cosine similarity matrix...
Done computing similarity matrix.
RMSE: 1.0097
RMSE (KNN): 1.0097036421415866
 * Running on https://e062-34-138-170-106.ngrok-free.app
 * Serving Flask app '__main__'
 * Debug mode: off


INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkzeug:Press CTRL+C to quit
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:41:26] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:41:26] "GET /favicon.ico HTTP/1.1" 404 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:41:34] "GET /recommend?user_id=1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:45:41] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:54:23] "GET /recommend?user_id=1 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:54:29] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:54:36] "GET /recommend?user_id=5 HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:54:40] "GET / HTTP/1.1" 200 -
INFO:werkzeug:127.0.0.1 - - [06/Feb/2025 08:54:46] "GET /recommend?user_id=1 HTTP/1.1" 200 -


In [ ]:
from pyngrok import ngrok
ngrok.set_auth_token("Your Authtoken")
